In [1]:
# !pip install google-cloud-bigquery

In [2]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/Jordi/git/IronHack/Ironhack Data Analytics.json"

In [3]:
from google.cloud import bigquery
client = bigquery.Client()

In [4]:
def select(query):
    query_job = client.query(query)
    df=query_job.to_dataframe()
    return df
    

# Challenge 1

In [5]:
query = '''
    SELECT 
        authors.au_id AS AUTHOR_ID,
        authors.au_lname AS LAST_NAME,
        authors.au_fname AS FIRST_NAME,
        titles.title AS TITLE,
        pubs.pub_name AS PUBLISHER
    FROM 
        `ironhack-data-analytics.publications.authors` authors
    JOIN
        `ironhack-data-analytics.publications.titleauthor` auti
        ON authors.au_id = auti.au_id
    JOIN
        `ironhack-data-analytics.publications.titles` titles
        ON auti.title_id = titles.title_id
    JOIN
        `ironhack-data-analytics.publications.publishers` pubs
        ON pubs.pub_id = titles.pub_id
    ORDER BY
        authors.au_id
    '''
df = select(query)
df.head(10)

,AUTHOR_ID,LAST_NAME,FIRST_NAME,TITLE,PUBLISHER
0,172-32-1176,White,Johnson,Prolonged Data Deprivation: Four Case Studies,New Moon Books
1,213-46-8915,Green,Marjorie,The Busy Executive's Database Guide,Algodata Infosystems
2,213-46-8915,Green,Marjorie,You Can Combat Computer Stress!,New Moon Books
3,238-95-7766,Carson,Cheryl,But Is It User Friendly?,Algodata Infosystems
4,267-41-2394,O'Leary,Michael,Cooking with Computers: Surreptitious Balance ...,Algodata Infosystems
5,267-41-2394,O'Leary,Michael,"Sushi, Anyone?",Binnet & Hardley
6,274-80-9391,Straight,Dean,Straight Talk About Computers,Algodata Infosystems
7,409-56-7008,Bennet,Abraham,The Busy Executive's Database Guide,Algodata Infosystems
8,427-17-2319,Dull,Ann,Secrets of Silicon Valley,Algodata Infosystems
9,472-27-2349,Gringlesby,Burt,"Sushi, Anyone?",Binnet & Hardley


# Challenge 2

In [6]:
query = '''
    SELECT 
        authors.au_id AS AUTHOR_ID,
        authors.au_lname AS LAST_NAME,
        authors.au_fname AS FIRST_NAME,
        pubs.pub_name AS PUBLISHER,
        COUNT(titles.title) AS TITLE_COUNT
    FROM 
        `ironhack-data-analytics.publications.authors` authors
    JOIN
        `ironhack-data-analytics.publications.titleauthor` auti
        ON authors.au_id = auti.au_id
    JOIN
        `ironhack-data-analytics.publications.titles` titles
        ON auti.title_id = titles.title_id
    JOIN
        `ironhack-data-analytics.publications.publishers` pubs
        ON pubs.pub_id = titles.pub_id
    GROUP BY 
        1,2,3,4
    ORDER BY
        COUNT(titles.title) DESC, 
        authors.au_id DESC
    '''
df = select(query)
df.head(10)

,AUTHOR_ID,LAST_NAME,FIRST_NAME,PUBLISHER,TITLE_COUNT
0,998-72-3567,Ringer,Albert,New Moon Books,2
1,899-46-2035,Ringer,Anne,New Moon Books,1
2,899-46-2035,Ringer,Anne,Binnet & Hardley,1
3,846-92-7186,Hunter,Sheryl,Algodata Infosystems,1
4,807-91-6654,Panteley,Sylvia,Binnet & Hardley,1
5,756-30-7391,Karsen,Livia,Binnet & Hardley,1
6,724-80-9391,MacFeather,Stearns,Binnet & Hardley,1
7,724-80-9391,MacFeather,Stearns,Algodata Infosystems,1
8,722-51-5454,DeFrance,Michel,Binnet & Hardley,1
9,712-45-1867,del Castillo,Innes,Binnet & Hardley,1


# Challenge 3

In [7]:
query = '''
    SELECT 
        authors.au_id AS AUTHOR_ID,
        authors.au_lname AS LAST_NAME,
        authors.au_fname AS FIRST_NAME,
        SUM(sales.qty) AS TOTAL
    FROM 
        `ironhack-data-analytics.publications.authors` authors
    JOIN
        `ironhack-data-analytics.publications.titleauthor` auti
        ON authors.au_id = auti.au_id
    JOIN
        `ironhack-data-analytics.publications.titles` titles
        ON auti.title_id = titles.title_id
    JOIN
        `ironhack-data-analytics.publications.sales` sales
        ON sales.title_id = titles.title_id
    GROUP BY 
        1,2,3
    ORDER BY
        SUM(sales.qty) DESC,
        authors.au_id DESC
    LIMIT 3
    '''
df = select(query)
df.head(10)

,AUTHOR_ID,LAST_NAME,FIRST_NAME,TOTAL
0,899-46-2035,Ringer,Anne,148
1,998-72-3567,Ringer,Albert,133
2,846-92-7186,Hunter,Sheryl,50


# Challenge 4

In [8]:
query = '''
    SELECT 
        authors.au_id AS AUTHOR_ID,
        authors.au_lname AS LAST_NAME,
        authors.au_fname AS FIRST_NAME,
        COALESCE(SUM(sales.qty),0) AS TOTAL
    FROM 
        `ironhack-data-analytics.publications.authors` authors
    FULL OUTER JOIN
        `ironhack-data-analytics.publications.titleauthor` auti
        ON authors.au_id = auti.au_id
    FULL OUTER JOIN
        `ironhack-data-analytics.publications.sales` sales
        ON sales.title_id = auti.title_id
    GROUP BY 
        1,2,3
    ORDER BY
        SUM(sales.qty) DESC,
        authors.au_id DESC

    '''
df = select(query)
df.head(25)

,AUTHOR_ID,LAST_NAME,FIRST_NAME,TOTAL
0,899-46-2035,Ringer,Anne,148
1,998-72-3567,Ringer,Albert,133
2,846-92-7186,Hunter,Sheryl,50
3,427-17-2319,Dull,Ann,50
4,213-46-8915,Green,Marjorie,50
5,724-80-9391,MacFeather,Stearns,45
6,267-41-2394,O'Leary,Michael,45
7,807-91-6654,Panteley,Sylvia,40
8,722-51-5454,DeFrance,Michel,40
9,238-95-7766,Carson,Cheryl,30


# Bonus Challenge
#### Nota: no he encontrado la columna 'advance' en toda la BD, según el enunciado había que sumarlo.

In [9]:
query = '''
    WITH royalties AS (
    SELECT 
        authors.au_id AS AUTHOR_ID,
        titles.title AS TITLE,
        titles.price * sales.qty * titles.royalty / 100 * titleauthor.royaltyper / 100 AS ROYALTY
    FROM 
        `ironhack-data-analytics.publications.authors` authors
    JOIN
        `ironhack-data-analytics.publications.titleauthor` titleauthor
        ON authors.au_id = titleauthor.au_id
    JOIN
        `ironhack-data-analytics.publications.titles` titles
        ON titleauthor.title_id = titles.title_id
    JOIN
        `ironhack-data-analytics.publications.sales` sales
        ON sales.title_id = titles.title_id
    )
    SELECT 
        authors.au_id AS AUTHOR_ID,
        authors.au_lname AS LAST_NAME,
        authors.au_fname AS FIRST_NAME,
        COALESCE(SUM(royalties.ROYALTY),0) AS PROFIT
    FROM 
        `ironhack-data-analytics.publications.authors` authors
    FULL OUTER JOIN
        `ironhack-data-analytics.publications.titleauthor` auti
        ON authors.au_id = auti.au_id
    FULL OUTER JOIN
        `ironhack-data-analytics.publications.sales` sales
        ON sales.title_id = auti.title_id
    JOIN 
        royalties 
        ON AUTHOR_ID = AUTHOR_ID
    GROUP BY 
        1,2,3
    ORDER BY
        SUM(royalties.ROYALTY) DESC,
        authors.au_id DESC
    LIMIT 3

    '''
df = select(query)
df.head(25)

,AUTHOR_ID,LAST_NAME,FIRST_NAME,PROFIT
0,899-46-2035,Ringer,Anne,4665.630
1,998-72-3567,Ringer,Albert,3888.025
2,213-46-8915,Green,Marjorie,2332.815
